<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/graphtrek_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
# install necessary modules
!pip install yfinance
!pip install plotly==5.2.1
!pip install ta
!pip install mplfinance

In [142]:
import pandas as pd
import yfinance as yf

from datetime import datetime, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import requests
from bs4 import BeautifulSoup
import numpy as np
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt


from ta.trend import MACD
from ta.momentum import StochasticOscillator
from ta.momentum import RSIIndicator

from operator import itemgetter

In [185]:
ticker = yf.Ticker("TSLA")
ticker.splits
min_strike = 10
#historical = ticker.history(start="2020-02-05", end="2020-02-25", interval="1d")
#historical

In [186]:
# method 1: fractal candlestick pattern

def get_stock_price(ticker):
  #df = yf.download(ticker.ticker, start='2020-01-01')
  #df = df.rename(columns={"Close": "Close1", "Adj Close": "Close"})
  
  #ticker = yf.Ticker(symbol)

  df = ticker.history(start="2020-12-01", interval="1d")
  #print(df.info())
  df['Date'] = pd.to_datetime(df.index)
  df['Date'] = df['Date'].apply(mpl_dates.date2num)
  #df = df.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]
  df['MA50'] = df['Close'].rolling(window=50).mean()
  df['MA100'] = df['Close'].rolling(window=100).mean()
  return df

def is_support(df,i):
  cond1 = df['Low'][i] < df['Low'][i-1] 
  cond2 = df['Low'][i] < df['Low'][i+1] 
  cond3 = df['Low'][i+1] < df['Low'][i+2] 
  cond4 = df['Low'][i-1] < df['Low'][i-2]
  return (cond1 and cond2 and cond3 and cond4)

def is_resistance(df,i):
  cond1 = df['High'][i] > df['High'][i-1] 
  cond2 = df['High'][i] > df['High'][i+1] 
  cond3 = df['High'][i+1] > df['High'][i+2] 
  cond4 = df['High'][i-1] > df['High'][i-2]
  return (cond1 and cond2 and cond3 and cond4)

def is_far_from_level(value, levels, df):
    ave =  np.mean(df['High'] - df['Low'])
    return np.sum([abs(value - level) < ave for _, level in levels]) == 0

def plot_all(levels, df):
    fig, ax = plt.subplots(figsize=(16, 9), dpi=300)
    candlestick_ohlc(ax,df.values,width=0.6, colorup='green', colordown='red', alpha=0.8)
    date_format = mpl_dates.DateFormatter('%d %b %Y')
    ax.xaxis.set_major_formatter(date_format)
    for level in levels:
        plt.hlines(level[1], xmin=df['Date'][level[0]], xmax=max(df['Date']), colors='blue', linestyle='--')
    fig.show()


def options_chain(ticker):

    #tk = yf.Ticker(symbol)
    # Expiration dates
    exps = ticker.options

    # Get options for each expiration
    options = pd.DataFrame()
    for e in exps:
        opt = ticker.option_chain(e)
        opt = pd.DataFrame().append(opt.calls).append(opt.puts)
        opt['expirationDate'] = e
        options = options.append(opt, ignore_index=True)

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    #options['expirationDate'] = pd.date(options['expirationDate']) + timedelta(days = 1)
    #options['dte'] = (options['expirationDate'] - datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: "C" in x)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    
    # Drop unnecessary and meaningless columns
    #options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [187]:
#symbol = 'MU'
df = get_stock_price(ticker)
levels = []
for i in range(2,len(df)-2):
  if is_support(df,i):
    l = df['Low'][i]
    if is_far_from_level(l, levels, df):
      levels.append((i,l))
  elif is_resistance(df,i):
    l = df['High'][i]
    if is_far_from_level(l, levels, df):
      levels.append((i,l))
levels = sorted(levels, key=itemgetter(1), reverse=True)

#plot_all(levels, df)

In [188]:
# MACD
macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
# stochastics
stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)

rsi = RSIIndicator(close=df['Close'], window=14)

In [189]:
# add subplot properties when initiliazing fig variable
fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.5,0.1,0.2,0.2])
# Plot OHLC on 1st subplot (using the codes from before)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close'], 
                             showlegend=False))
# add moving average traces
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         line=dict(color='blue', width=2), 
                         name='MA 50'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df['MA100'], 
                         line=dict(color='orange', width=2), 
                         name='MA 100'))


for idx, level in  enumerate(levels):
    percent = 0
    current_level = level[1]
    if idx > 0:
      prev_level = levels[idx-1][1]
      diff = prev_level - current_level
      percent = (diff / current_level) * 100
      #print(percent)
    fig.add_hline(level[1], annotation_position="top left", annotation_text=str(np.round(current_level,1)) + ' (' + str(np.round(percent,1)) + '%)' )

# Plot volume trace on 2nd row 
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors
                    ), row=2, col=1)

# Plot MACD trace on 3rd row
colors = ['green' if val >= 0 
          else 'red' for val in macd.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     marker_color=colors
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)


fig.add_trace(go.Scatter(x=df.index,
                         y=rsi.rsi(),
                         line=dict(color='black', width=2)
                        ), row=4, col=1)


# Plot stochastics trace on 4th row 
#fig.add_trace(go.Scatter(x=df.index,
#                         y=stoch.stoch(),
#                         line=dict(color='black', width=2)
#                        ), row=4, col=1)
#fig.add_trace(go.Scatter(x=df.index,
#                         y=stoch.stoch_signal(),
#                         line=dict(color='blue', width=1)
#                        ), row=4, col=1)


# removing rangeslider
fig.update_layout(xaxis_rangeslider_visible=False)

# removing all empty dates
# build complete timeline from start date to end date
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

fig.update_layout(xaxis_rangebreaks=[dict(values=dt_breaks)])
fig.update_layout(margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20  #top margin
    ))

# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])

# update y-axis label
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)
fig.update_yaxes(showspikes=True, spikemode='across', spikesnap='cursor',spikedash='dash')
fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor', spikedash='dash')
fig.show()

In [190]:
options_df = options_chain(ticker)
options_df1 =  options_df.loc[options_df['strike'] > min_strike]

column = options_df1["openInterest"]
max_value = column.max()
max_index = column.idxmax()
print('max_value',max_value,'max_index',max_index)

options_df1.loc[max_index]

max_value 39263.0 max_index 1514


contractSymbol       TSLA220121P00020000
lastTradeDate        2021-12-10 15:36:18
strike                                20
lastPrice                           0.01
bid                                    0
ask                                 0.01
change                                 0
percentChange                          0
volume                                20
openInterest                       39263
impliedVolatility                 3.4375
inTheMoney                         False
contractSize                     REGULAR
currency                             USD
expirationDate                2022-01-21
CALL                               False
mark                               0.005
Name: 1514, dtype: object